In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def create_toy_data(func, sample_size, std):
    x = np.linspace(0, 1, sample_size)
    t = func(x) + np.random.normal(scale=std, size=x.shape)
    return x, t

def func(x):
    return np.sin(2 * np.pi * x)

x_train, y_train = create_toy_data(func, 10, 0.25)
x_test = np.linspace(0, 1, 100)
y_test = func(x_test)

(a) Plot the graph with given code, the result should be same as this.
![](originalData.png)
`x_train` and `y_train` are the datas you need to create, `sample_size` is 10 and `std` is 0.25. 

In [ ]:
plt.plot(x_test, y_test, label='sin(2πx)', color='green')
plt.scatter(x_train, y_train, label='training data', c='none', edgecolors='blue')

# plt.xticks(np.arange(0, 1.1, 0.2))
# plt.yticks(np.arange(-1, 1.3, 0.5))

plt.legend()
plt.show()

(b) On the basis of the results, you should try $0^{th}$ order polynomial, $1^{st}$ order polynomial, $3^{rd}$ order polynomial and some other order polynomial, show the results include fitting and over-fitting.
![](fitting.png)

In [ ]:
import itertools
import functools

class PolynomialFeature(object):
    """
    polynomial features

    transforms input array with polynomial features

    Example
    =======
    x =
    [[a, b],
    [c, d]]

    y = PolynomialFeatures(degree=2).transform(x)
    y =
    [[1, a, b, a^2, a * b, b^2],
    [1, c, d, c^2, c * d, d^2]]
    """

    def __init__(self, degree=2):
        """
        construct polynomial features

        Parameters
        ----------
        degree : int
            degree of polynomial
        """
        assert isinstance(degree, int)
        self.degree = degree

    def transform(self, x):
        """
        transforms input array with polynomial features

        Parameters
        ----------
        x : (sample_size, n) ndarray
            input array

        Returns
        -------
        output : (sample_size, 1 + nC1 + ... + nCd) ndarray
            polynomial features
        """
        if x.ndim == 1:
            x = x[:, None]
        x_t = x.transpose()
        features = [np.ones(len(x))]
        for degree in range(1, self.degree + 1):
            for items in itertools.combinations_with_replacement(x_t, degree):
                features.append(functools.reduce(lambda x, y: x * y, items))
        return np.asarray(features).transpose()
    
class Regression(object):
    """
    Base class for regressors
    """
    pass

class LinearRegression(Regression):
    """
    Linear regression model
    y = X @ w
    t ~ N(t|X @ w, var)
    """

    def fit(self, X:np.ndarray, t:np.ndarray):
        """
        perform least squares fitting

        Parameters
        ----------
        X : (N, D) np.ndarray
            training independent variable
        t : (N,) np.ndarray
            training dependent variable
        """
        self.w = np.linalg.pinv(X) @ t
        self.var = np.mean(np.square(X @ self.w - t))

    def predict(self, X:np.ndarray, return_std:bool=False):
        """
        make prediction given input

        Parameters
        ----------
        X : (N, D) np.ndarray
            samples to predict their output
        return_std : bool, optional
            returns standard deviation of each predition if True

        Returns
        -------
        y : (N,) np.ndarray
            prediction of each sample
        y_std : (N,) np.ndarray
            standard deviation of each predition
        """
        y = X @ self.w
        if return_std:
            y_std = np.sqrt(self.var) + np.zeros_like(y)
            return y, y_std
        return y

In [ ]:
from typing import Tuple


def plot_polynomial(x_train: np.array, y_train: np.array,
          x_test: np.array, y_test: np.array,
          degree: int, index: Tuple[int, int, int], weights: map=None):
    pf = PolynomialFeature(degree)
    feat_train, feat_test = pf.transform(x_train), pf.transform(x_test)
    model = LinearRegression()
    model.fit(feat_train, y_train)
    y_pred = model.predict(feat_test)
    if weights is not None:
        weights[degree] = model.w[-1]
    plt.subplot(*index)
    plt.title(f'{degree} order polynomial, {len(x_train)} samples')
    plt.plot(x_test, y_test, label='sin(2πx)', color='green')
    plt.scatter(x_train, y_train, label='training data', c='none', edgecolors='blue')
    plt.plot(x_test, y_pred, label=f'{degree}-order pred', color='red')
    plt.legend()

plt.figure(figsize=(12, 8))
plot_polynomial(x_train, y_train, x_test, y_test, 0, (2, 2, 1))
plot_polynomial(x_train, y_train, x_test, y_test, 1, (2, 2, 2))
plot_polynomial(x_train, y_train, x_test, y_test, 3, (2, 2, 3))
plot_polynomial(x_train, y_train, x_test, y_test, 10, (2, 2, 4))
plt.show()

(c) Plot the graph of the root-mean-square error.
![](rmse.png)

In [ ]:
from typing import Tuple

def rmse(a, b):
    mse = np.mean(np.square(a - b))
    return np.sqrt(mse)

def train_test_rmse(x_train, y_train, x_test, y_test, degree) -> Tuple[float, float]:
    pf = PolynomialFeature(degree)
    feat_train, feat_test = pf.transform(x_train), pf.transform(x_test)
    model = LinearRegression()
    model.fit(feat_train, y_train)
    y_train_pred = model.predict(feat_train)
    y_test_pred = model.predict(feat_test)
    return rmse(y_train, y_train_pred), rmse(y_test, y_test_pred)

In [ ]:
training_errors = []
test_errors = []

for degree in range(11):
    rmse_train, rmse_test = train_test_rmse(x_train, y_train, x_test, y_test, degree)
    training_errors.append(rmse_train)
    test_errors.append(rmse_test)
    
plt.scatter(range(11), training_errors, label='train')
plt.plot(range(11), training_errors)

plt.scatter(range(11), test_errors, label='test')
plt.plot(range(11), test_errors)

plt.xlabel('degree')
plt.ylabel('RMSE')
plt.legend()
plt.show()

(d) Plot the graph of the predictive distribution resulting from a Bayesian treatment of polynomial curve fitting using an M=9 polynomial, with the fixed parameters $\alpha=5\times 10^{-3}$ and $\beta=11.1$(corresponding to the known noise variance).
![](bayesianRegression.png)

In [ ]:
class BayesianRegression(Regression):
    """
    Bayesian regression model

    w ~ N(w|0, alpha^(-1)I)
    y = X @ w
    t ~ N(t|X @ w, beta^(-1))
    """

    def __init__(self, alpha:float=1., beta:float=1.):
        self.alpha = alpha
        self.beta = beta
        self.w_mean = None
        self.w_precision = None

    def _is_prior_defined(self) -> bool:
        return self.w_mean is not None and self.w_precision is not None

    def _get_prior(self, ndim:int) -> tuple:
        if self._is_prior_defined():
            return self.w_mean, self.w_precision
        else:
            return np.zeros(ndim), self.alpha * np.eye(ndim)

    def fit(self, X:np.ndarray, t:np.ndarray):
        """
        bayesian update of parameters given training dataset

        Parameters
        ----------
        X : (N, n_features) np.ndarray
            training data independent variable
        t : (N,) np.ndarray
            training data dependent variable
        """

        mean_prev, precision_prev = self._get_prior(np.size(X, 1))

        w_precision = precision_prev + self.beta * X.T @ X
        w_mean = np.linalg.solve(
            w_precision,
            precision_prev @ mean_prev + self.beta * X.T @ t
        )
        self.w_mean = w_mean
        self.w_precision = w_precision
        self.w_cov = np.linalg.inv(self.w_precision)

    def predict(self, X:np.ndarray, return_std:bool=False, sample_size:int=None):
        """
        return mean (and standard deviation) of predictive distribution

        Parameters
        ----------
        X : (N, n_features) np.ndarray
            independent variable
        return_std : bool, optional
            flag to return standard deviation (the default is False)
        sample_size : int, optional
            number of samples to draw from the predictive distribution
            (the default is None, no sampling from the distribution)

        Returns
        -------
        y : (N,) np.ndarray
            mean of the predictive distribution
        y_std : (N,) np.ndarray
            standard deviation of the predictive distribution
        y_sample : (N, sample_size) np.ndarray
            samples from the predictive distribution
        """

        if sample_size is not None:
            w_sample = np.random.multivariate_normal(
                self.w_mean, self.w_cov, size=sample_size
            )
            y_sample = X @ w_sample.T
            return y_sample
        y = X @ self.w_mean
        if return_std:
            y_var = 1 / self.beta + np.sum(X @ self.w_cov * X, axis=1)
            y_std = np.sqrt(y_var)
            return y, y_std
        return y

In [ ]:
pf = PolynomialFeature(9)
feat_train, feat_test = pf.transform(x_train), pf.transform(x_test)

bayes = BayesianRegression(alpha=5*10**-3, beta=11.1)
bayes.fit(feat_train, y_train)
y_pred, y_std = bayes.predict(feat_test, return_std=True)

plt.plot(x_test, y_test, label='sin(2πx)')
plt.plot(x_test, y_pred, label='bayes')
plt.scatter(x_train, y_train, label='training data', c='none', edgecolors='gray')
plt.fill_between(x_test, y_pred - y_std, y_pred + y_std, alpha=0.2)

plt.legend()
plt.show()

(e) Change the $sample\_size$ to 2, 3 or 10 times than before, explain the change of $M$.

In [ ]:
def plot(func, sample_size, w):
    x_train, y_train = create_toy_data(func, sample_size, 0.25)
    x_test = np.linspace(0, 1, 100)
    y_test = func(x_test)

    plt.figure(figsize=(16, 3))
    plot_polynomial(x_train, y_train, x_test, y_test, 0, (1, 4, 1), w)
    plot_polynomial(x_train, y_train, x_test, y_test, 1, (1, 4, 2), w)
    plot_polynomial(x_train, y_train, x_test, y_test, 3, (1, 4, 3), w)
    plot_polynomial(x_train, y_train, x_test, y_test, 10, (1, 4, 4), w)
    plt.show()

w20, w30, w100 = {}, {}, {}
plot(func, 20, w20)
plot(func, 30, w30)
plot(func, 100, w100)

In [ ]:
from cProfile import label


m0 = {20: w20[0], 30: w30[0], 100: w100[0]}
m1 = {20: w20[1], 30: w30[1], 100: w100[1]}
m3 = {20: w20[3], 30: w30[3], 100: w100[3]}
m10 = {20: w20[10], 30: w30[10], 100: w100[10]}

display(m0.values(), m1.values(), m3.values(), m10.values())

plt.figure(figsize=(4, 3))
plt.plot(m0.keys(), m0.values(), marker='p', label='M = 0')
plt.plot(m1.keys(), m1.values(), marker='p', label='M = 1')
plt.plot(m3.keys(), m3.values(), marker='p', label='M = 3')
plt.plot(m10.keys(), m10.values(), marker='p', label='M = 10')

plt.xlabel('sample size')
plt.ylabel('max norm of w')
plt.legend()
plt.show()

From the above regressions, we can easily find out that:
1. with $M$ increasing, it's more possible that the overfitting will be more serious
2. with sample size increasing, it can depress the overfitting, yet can't avoid it
3. larger sample size combined with appropriate degree can guide the trained model closer to actual distribution of the data
4. with fixed sample data, the accuracy of trained model saturated with certain $M$ and larger degree will not bring benifit to accuracy but only bring overfitting